In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Nov 22 12:56:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P5000        Off  | 00000000:00:05.0 Off |                  Off |
| 26%   27C    P8     6W / 180W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
from huggingface_hub import notebook_login


In [15]:
# notebook_login()

`jiwer` for Word Error Rate Metric.

In [6]:
%%capture
!pip install datasets==1.18.3
!pip install transformers==4.17.0
!pip install jiwer

In [7]:
!pip install soundfile

In [8]:
!pip install librosa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 50.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 45.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 56.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 35.2 MB/s eta 0:00:0000:0100:01
  Created wheel for audioread: filename=audioread-3.0.0-py3-none-any.whl size=23702 sha256=437a88038f9d6a1e4b46f8372cf64e8440c34d2a10ac52bb0dfda6e3da7a4edd
  Stored in directory: /root/.cache/pip/wheels/e4/76/a4/cfb55573167a1f5bde7d7a348e95e509c64b2c3e8f921932c3
Successfully built audioread


In [9]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash

Detected operating system as Ubuntu/focal.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.0.9
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.


In [10]:
!sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 103 not upgraded.
Need to get 7168 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu focal/main amd64 git-lfs amd64 3.2.0 [7168 kB]
Fetched 7168 kB in 1s (12.3 MB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 78560 files and directories currently installed.)
Preparing to unpack .../git-lfs_3.2.0_amd64.deb ...
Unpacking git-lfs (3.2.0) ...
Setting up git-lfs (3.2.0) ...
Git LFS initialized.
Processing triggers for man-db (2.9.1-1) ...


For Keeping records of Model Checkpoint
i.e MLOPs like thing

In [11]:
# from huggingface_hub import notebook_login

# notebook_login()

In [12]:
!apt-get install --yes git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (3.2.0).
0 upgraded, 0 newly installed, 0 to remove and 103 not upgraded.


Downloading the Timit Automatic Speech Recoginition Dataset. 

In [13]:
from datasets import load_dataset, load_metric

timit = load_dataset("timit_asr")

Downloading:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/869M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset timit_asr downloaded and prepared to /root/.cache/huggingface/datasets/timit_asr/clean/2.0.1/b11b576ddcccbcefa7c9f0c4e6c2a43756f3033adffe0fb686aa61043d0450ad. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
timit

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 1680
    })
})

Remove Columns that are not needed for ASR task

In [15]:
timit = timit.remove_columns(["phonetic_detail", "word_detail", "dialect_region", "id", "sentence_type", "speaker_id"])

In [16]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [17]:
show_random_elements(timit["train"].remove_columns(["audio", "file"]), num_examples=10)

,text
0,The prowler wore a ski mask for disguise.
1,"The groundhog clearly saw his shadow, but stayed out only a moment."
2,We apply auditory modeling to computer speech recognition.
3,She had your dark suit in greasy wash water all year.
4,Is there any word you would like to offer in your own defense?
5,"The dark, murky lagoon wound around for miles."
6,The cow wandered from the farmland and became lost.
7,Don't ask me to carry an oily rag like that.
8,The most recent geological survey found seismic activity.
9,The speech symposium might begin Monday.


In [18]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [19]:
timit = timit.map(remove_special_characters)

Parameter 'function'=<function remove_special_characters at 0x7fd32d540040> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [20]:
show_random_elements(timit["train"].remove_columns(["audio", "file"]))

,text
0,they offered no opinions volunteered nothing betrayed no emotions
1,don't ask me to carry an oily rag like that
2,she had your dark suit in greasy wash water all year
3,orange juice tastes funny after toothpaste
4,she smiled and teeth gleamed in her beautifully modeled olive face
5,she had your dark suit in greasy wash water all year
6,but that heritage is shrinking even faster
7,topography topography is very important
8,don't ask me to carry an oily rag like that
9,smash lightbulbs and their cash value will diminish to nothing


CTC commonly make speech chunks to letters. In order to make speech text string to characters we are using following peice of code

In [21]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [22]:
vocabs = timit.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=timit.column_names["train"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Getting total Character (Union of Character in Train and Test Vocab)

In [23]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

In [24]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'q': 0,
 ' ': 1,
 'y': 2,
 'i': 3,
 'g': 4,
 'm': 5,
 'a': 6,
 'r': 7,
 "'": 8,
 'v': 9,
 'o': 10,
 'e': 11,
 'l': 12,
 's': 13,
 'n': 14,
 'j': 15,
 'z': 16,
 'f': 17,
 'c': 18,
 'x': 19,
 'u': 20,
 'k': 21,
 'b': 22,
 'h': 23,
 'd': 24,
 'w': 25,
 'p': 26,
 't': 27}

To make it clearer that " " has its own token class, we give it a more visible character |.

In [25]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

Added Padding and Unknown Label in Vocab. Blank/Padding Token is core component of CTC model

In [26]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

30

In [27]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [28]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [30]:
repo_name = "m-aliabbas/wav2vec2-base-timit-demo-idrak-practice"

In [31]:
tokenizer.push_to_hub(repo_name,use_auth_token=True)

/usr/local/lib/python3.9/dist-packages/huggingface_hub/utils/_deprecation.py:39: FutureWarning: Pass token='wav2vec2-base-timit-demo-idrak-practice' as keyword args. From version 0.8 passing these as positional arguments will result in an error
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/huggingface_hub/hf_api.py:641: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  warnings.warn(
Cloning https://huggingface.co/m-aliabbas/wav2vec2-base-timit-demo-idrak-practice into local empty directory.
To https://huggingface.co/m-aliabbas/wav2vec2-base-timit-demo-idrak-practice
   e26d878..3498316  main -> main



'https://huggingface.co/m-aliabbas/wav2vec2-base-timit-demo-idrak-practice/commit/3498316053decc9289b7804bc704bb00b1405617'

A Wav2Vec2 feature extractor object requires the following parameters to be instantiated:

- `feature_size`: Speech models take a sequence of feature vectors as an input. While the length of this sequence obviously varies, the feature size should not. In the case of Wav2Vec2, the feature size is 1 because the model was trained on the raw speech signal ${}^2$.
- `sampling_rate`: The sampling rate at which the model is trained on.
- `padding_value`: For batched inference, shorter inputs need to be padded with a specific value
- `do_normalize`: Whether the input should be *zero-mean-unit-variance* normalized or not. Usually, speech models perform better when normalizing the input
- `return_attention_mask`: Whether the model should make use of an `attention_mask` for batched inference. In general, models should **always** make use of the `attention_mask` to mask padded tokens. However, due to a very specific design choice of `Wav2Vec2`'s "base" checkpoint, better results are achieved when using no `attention_mask`. This is **not** recommended for other speech models. For more information, one can take a look at [this](https://github.com/pytorch/fairseq/issues/3227) issue. **Important** If you want to use this notebook to fine-tune [large-lv60](https://huggingface.co/facebook/wav2vec2-large-lv60), this parameter should be set to `True`.

In [32]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [33]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [34]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(timit["train"]))

print(timit["train"][rand_int]["text"])
ipd.Audio(data=np.asarray(timit["train"][rand_int]["audio"]["array"]), autoplay=True, rate=16000)

they all like long hot showers 


In [36]:
for i in range(10):
    rand_int = random.randint(0, len(timit["train"]))

    print("Target text:", timit["train"][rand_int]["text"])
    print("Input array shape:", np.asarray(timit["train"][rand_int]["audio"]["array"]).shape)
    print("Sampling rate:", timit["train"][rand_int]["audio"]["sampling_rate"])

Target text: the bluejay flew over the high building 
Input array shape: (34407,)
Sampling rate: 16000
Target text: cut a small corner off each edge 
Input array shape: (27444,)
Sampling rate: 16000
Target text: add remaining ingredients and bring to a boil 
Input array shape: (48436,)
Sampling rate: 16000
Target text: she had your dark suit in greasy wash water all year 
Input array shape: (46080,)
Sampling rate: 16000
Target text: don't ask me to carry an oily rag like that 
Input array shape: (42189,)
Sampling rate: 16000
Target text: may i order a strawberry sundae after i eat dinner 
Input array shape: (47412,)
Sampling rate: 16000
Target text: no antigen was detectable in certain dark spherical areas in most cells 
Input array shape: (71988,)
Sampling rate: 16000
Target text: scholastic aptitude is judged by standardized tests 
Input array shape: (51917,)
Sampling rate: 16000
Target text: steph could barely handle the psychological trauma 
Input array shape: (56935,)
Sampling rat

Note: This mapping function is a good example of how the Wav2Vec2Processor class should be used. In "normal" context, calling processor(...) is redirected to Wav2Vec2FeatureExtractor's call method. When wrapping the processor into the as_target_processor context, however, the same method is redirected to Wav2Vec2CTCTokenizer's call method. For more information please check the docs.

In [37]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [38]:
timit = timit.map(prepare_dataset, remove_columns=timit.column_names["train"], num_proc=4)

Remove the signal more than 20 sec

In [40]:
max_input_length_in_sec = 20.0
timit["train"] = timit["train"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

  0%|          | 0/5 [00:00<?, ?ba/s]

## Training & Evaluation

The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) for which we essentially need to do the following:

- Define a data collator. In contrast to most NLP models, Wav2Vec2 has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning Wav2Vec2 requires a special padding data collator, which we will define below

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

- Define the training configuration.

After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech.

In [41]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [42]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [43]:
wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

In [44]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [45]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

/usr/local/lib/python3.9/dist-packages/transformers/configuration_utils.py:356: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'project_q.bias', 'quantizer.weight_proj.bias', 'project_hid.bias', 'quantizer.weight_proj.weight', 'project_hid.weight', 'project_q.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

The first component of Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretrainind and as stated in the [paper](https://arxiv.org/abs/2006.11477) does not need to be fine-tuned anymore. 
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [46]:
model.freeze_feature_encoder()

In a final step, we define all parameters related to training. 
To give more explanation on some of the parameters:
- `group_by_length` makes training more efficient by grouping training samples of similar input length into one batch. This can significantly speed up training time by heavily reducing the overall number of useless padding tokens that are passed through the model
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Timit dataset and might be suboptimal for other speech datasets.

For more explanations on other parameters, one can take a look at the [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments).

During training, a checkpoint will be uploaded asynchronously to the hub every 400 training steps. It allows you to also play around with the demo widget even while your model is still training.

**Note**: If one does not want to upload the model checkpoints to the hub, simply set `push_to_hub=False`.

In [47]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=10,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

In [48]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=timit["train"],
    eval_dataset=timit["test"],
    tokenizer=processor.feature_extractor,
)

Using amp half precision backend


In [49]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4620
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5780


Step,Training Loss,Validation Loss,Wer
500,3.634000,2.645220,1.000000
1000,1.049700,0.571106,0.513817
1500,0.458400,0.442108,0.449245
2000,0.319800,0.381834,0.394115
2500,0.226300,0.365272,0.376680
3000,0.184500,0.342423,0.366136
3500,0.138800,0.370193,0.351940
4000,0.121400,0.351509,0.343877
4500,0.102600,0.358456,0.329199
5000,0.083400,0.347395,0.323617


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1680
  Batch size = 8
Saving model checkpoint to m-aliabbas/wav2vec2-base-timit-demo-idrak-practice/checkpoint-500
Configuration saved in m-aliabbas/wav2vec2-base-timit-demo-idrak-practice/checkpoint-500/config.json
Model weights saved in m-aliabbas/wav2vec2-base-timit-demo-idrak-practice/checkpoint-500/pytorch_model.bin
Feature extractor saved in m-aliabbas/wav2vec2-base-timit-demo-idrak-practice/checkpoint-500/preprocessor_config.json
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluat

TrainOutput(global_step=5780, training_loss=0.5562470601091748, metrics={'train_runtime': 4103.3192, 'train_samples_per_second': 11.259, 'train_steps_per_second': 1.409, 'total_flos': 1.3094724388304794e+18, 'train_loss': 0.5562470601091748, 'epoch': 10.0})

In [50]:
trainer.push_to_hub()

/notebooks/m-aliabbas/wav2vec2-base-timit-demo-idrak-practice is already a clone of https://huggingface.co/m-aliabbas/wav2vec2-base-timit-demo-idrak-practice. Make sure you pull the latest changes with `repo.git_pull()`.
Saving model checkpoint to m-aliabbas/wav2vec2-base-timit-demo-idrak-practice
Configuration saved in m-aliabbas/wav2vec2-base-timit-demo-idrak-practice/config.json
Model weights saved in m-aliabbas/wav2vec2-base-timit-demo-idrak-practice/pytorch_model.bin
Feature extractor saved in m-aliabbas/wav2vec2-base-timit-demo-idrak-practice/preprocessor_config.json


Upload file pytorch_model.bin:   0%|          | 32.0k/360M [00:00<?, ?B/s]

Upload file runs/Nov22_13-04-57_nw48qbpnu2/events.out.tfevents.1669122413.nw48qbpnu2.32.0: 100%|##########| 10…

Upload file runs/Nov22_13-04-57_nw48qbpnu2/1669122413.1371827/events.out.tfevents.1669122413.nw48qbpnu2.32.1: …

Upload file training_args.bin: 100%|##########| 2.98k/2.98k [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/m-aliabbas/wav2vec2-base-timit-demo-idrak-practice
   3498316..51c1dce  main -> main

Dropping the following result as it does not have all the necessary fields:
{}
To https://huggingface.co/m-aliabbas/wav2vec2-base-timit-demo-idrak-practice
   51c1dce..46781a9  main -> main



'https://huggingface.co/m-aliabbas/wav2vec2-base-timit-demo-idrak-practice/commit/51c1dce12adcd928c261ebba0c2933d7d8fb1339'

In [45]:
# processor = Wav2Vec2Processor.from_pretrained("m-aliabbas/wav2vec2-base-timit-demo-idrak-paperspace")

loading feature extractor configuration file m-aliabbas/wav2vec2-base-timit-demo-idrak-paperspace/preprocessor_config.json
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

Didn't find file m-aliabbas/wav2vec2-base-timit-demo-idrak-paperspace/added_tokens.json. We won't load it.
loading file m-aliabbas/wav2vec2-base-timit-demo-idrak-paperspace/vocab.json
loading file m-aliabbas/wav2vec2-base-timit-demo-idrak-paperspace/tokenizer_config.json
loading file None
loading file m-aliabbas/wav2vec2-base-timit-demo-idrak-paperspace/special_tokens_map.json
Adding <s> to the vocabulary
Adding </s> to the vocabulary
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# model = Wav2Vec2ForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-google-colab").cuda()

In [71]:
repo_name

'm-aliabbas/wav2vec2-base-timit-demo-idrak-paperspace1'